# Business Analytics - Assignment 1 - Question 1
#### **Student Name:** Koorosh Shakoori

# Financial Portfolio

In this task we need to solve a linear programming problem using pulp.

In [2]:
from pulp import *

## Defining the decision variables
To define the decision variables we make a Funds list to iterate over and define the variables,and two dictionaries for Annual Return and Risk Measure for later purposes.

In [3]:
#First we define the funds and its features as lists and dictionaries
Funds = [
    'Low-Priced_Stock_Fund',
    'Multinational_Fund',
    'Mid-Cap_Stock_Fund',
    'Mortgage_Fund',
    'Income_Equity_Fund',
    'Balanced_Fund'
]

Annual_Return = {
    'Low-Priced_Stock_Fund': 8.13,
    'Multinational_Fund': 9.02,
    'Mid-Cap_Stock_Fund': 7.56,
    'Mortgage_Fund': 3.62,
    'Income_Equity_Fund': 7.79,
    'Balanced_Fund': 4.4
}

Risk_Measure = {
    'Low-Priced_Stock_Fund': 10.57,
    'Multinational_Fund': 13.22,
    'Mid-Cap_Stock_Fund': 14.02,
    'Mortgage_Fund': 2.39,
    'Income_Equity_Fund': 9.3,
    'Balanced_Fund': 7.61
}

#The decision variables in this problem are the amount of investment in each fund out of the six available shown in Funds list above.
#It is clear that we cannot invest negative amounts, hence, the lowbound is set to zero.
invest_variables = LpVariable.dicts('Invest', Funds, lowBound=0)

## Defining the Objective Function
Given the Risk measure of each fund, the problem asks for minimum weighted risk while meeting all the conditions.
Therefore the objective function is defined as below:

In [17]:
#Now we will intialize a LP problem object from pulp with the objective to minimize to fit our objective described below.
prob = LpProblem('Financial_Portfolio', LpMinimize)

#The objective function in this problem is the request of our client to minimize the weighted risk.
#For this problem to be linear, however, we will need to assume that we use all the 500k budget.
#If not, we need to replace the denominator of 500000 with summation of our variables, which will result in nonlinearity.
prob += lpSum([invest_variables[fund]*Risk_Measure[fund] for fund in Funds])/500000

## Defining Constraints
Now we set Constraints as required in the assignments, we need to address:
- the summation of all the funds is equal to 500000
- the least amount of 50000 in multinational and balanced funds
- the combined least income equity and balanced fund of 200000
- the minimum average return constraint
- the maximum of 200000 for each individual fund 

In [18]:
#With our objective function in place we now define the constraints accordingly
#First we have to make sure our total investment is equal to the 500K budget
prob += lpSum([invest_variables[fund] for fund in Funds]) == 500000, 'Budget_Limit'
#The next two constraints will limit the minimum of 50K investment in each of Multinational Fund and Balanced Fund as advised by the company
prob += invest_variables['Multinational_Fund'] >= 50000, 'Least_Multinational_Fund'
prob += invest_variables['Balanced_Fund'] >= 50000, 'Least_Balanced_Fund'
#We also need to satisfy the condition of the least combined investment in Income Equity Fund and Balanced Fund of 200k
prob += lpSum([invest_variables['Income_Equity_Fund'], invest_variables['Balanced_Fund']]) >= 200000, 'Least_Combined_Income_Equity+Balanced_Funds'
#Lastly we apply the constraint put by the customer to keep the average return rate of 5%`
prob += lpSum([invest_variables[fund]*Annual_Return[fund] for fund in Funds])/500000 >= 5, 'Average_Return_Constraint'
#Maximum Investment in individual funds
for fund in Funds:
    prob += invest_variables[fund] <= 200000, f'Maximum {fund} investment'

In [19]:
#Here we get an overall report on all sides of the problem
prob

Financial_Portfolio:
MINIMIZE
1.522e-05*Invest_Balanced_Fund + 1.86e-05*Invest_Income_Equity_Fund + 2.114e-05*Invest_Low_Priced_Stock_Fund + 2.804e-05*Invest_Mid_Cap_Stock_Fund + 4.78e-06*Invest_Mortgage_Fund + 2.644e-05*Invest_Multinational_Fund + 0.0
SUBJECT TO
Budget_Limit: Invest_Balanced_Fund + Invest_Income_Equity_Fund
 + Invest_Low_Priced_Stock_Fund + Invest_Mid_Cap_Stock_Fund
 + Invest_Mortgage_Fund + Invest_Multinational_Fund = 500000

Least_Multinational_Fund: Invest_Multinational_Fund >= 50000

Least_Balanced_Fund: Invest_Balanced_Fund >= 50000

Least_Combined_Income_Equity_Balanced_Funds: Invest_Balanced_Fund
 + Invest_Income_Equity_Fund >= 200000

Average_Return_Constraint: 8.8e-06 Invest_Balanced_Fund
 + 1.558e-05 Invest_Income_Equity_Fund
 + 1.626e-05 Invest_Low_Priced_Stock_Fund
 + 1.512e-05 Invest_Mid_Cap_Stock_Fund + 7.24e-06 Invest_Mortgage_Fund
 + 1.804e-05 Invest_Multinational_Fund >= 5

Maximum_Low_Priced_Stock_Fund_investment: Invest_Low_Priced_Stock_Fund
 <= 200

In [20]:
prob.solve(PULP_CBC_CMD(msg=0))

1

In [21]:
#The optimal portfolio is described as below
#The risk measure is also mentioned in last.
for var in prob.variables():
    print(f'{var.name} = {var.varValue}')
print(f'Weighted Risk of the portfolio = {prob.objective.value()}')

Invest_Balanced_Fund = 183628.32
Invest_Income_Equity_Fund = 66371.681
Invest_Low_Priced_Stock_Fund = 0.0
Invest_Mid_Cap_Stock_Fund = 0.0
Invest_Mortgage_Fund = 200000.0
Invest_Multinational_Fund = 50000.0
Weighted Risk of the portfolio = 6.307336297


In [22]:
import pandas as pd
#Report Sensitivity Analysis
print("\nSensitivity Analysis")
Cons_Sensitivity_Report=[{'Constraint_Name':name,'Slack':c.slack,"Shadow_Price":c.pi}
                         for name, c in prob.constraints.items()]
print(pd.DataFrame(Cons_Sensitivity_Report))


Sensitivity Analysis
                                Constraint_Name       Slack  Shadow_Price
0                                  Budget_Limit      -0.000      0.000011
1                      Least_Multinational_Fund      -0.000      0.000007
2                           Least_Balanced_Fund -133628.320      0.000000
3   Least_Combined_Income_Equity_Balanced_Funds  -50000.000      0.000000
4                     Average_Return_Constraint      -0.000      0.498525
5      Maximum_Low_Priced_Stock_Fund_investment  200000.000      0.000000
6         Maximum_Multinational_Fund_investment  150000.000      0.000000
7         Maximum_Mid_Cap_Stock_Fund_investment  200000.000      0.000000
8              Maximum_Mortgage_Fund_investment      -0.000     -0.000010
9         Maximum_Income_Equity_Fund_investment  133628.319      0.000000
10             Maximum_Balanced_Fund_investment   16371.680      0.000000


## Answers to question B ,C and, D
#### B.
**Mortgage Funds.** Given that this fund has the lowest risk rate, making it the only fund utilized to its full 200k maximum limit, and also considering its shadow price is the best choice for increasing the 200k limit while keeping the risk measure as low as possible.
#### C.
**Yes, It will increase the risk.** considering the apparent positive correlation between risk measure and annual return, increasing the return will inevitably elevate the overal risk.
#### D.
**False.** Since this fund has the highest risk measure, by decreasing it's minimum required investment, the allocated fund will be replaced with lower risk funds, in turn lessening the overall weighted risk factor.